# PYCUDA (GPU) version.

You may try when you have available PYCUDA (GPU).

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# ### All Import Statements and Confirming GPUs are Connected

# In[14]:


import os, os.path
from osgeo import gdal
from osgeo.gdalconst import *
from matplotlib import pyplot as plt
import numpy as np
import rasterio
import time
import math
import sys

from pycuda.compiler import SourceModule
import pycuda
from pycuda import gpuarray
from pycuda import compiler
import pycuda.driver as cuda
import pycuda.autoinit             # PyCuda autoinit
import pycuda.driver as cuda       # PyCuda In, Out helpers
import matplotlib.pyplot as plot   # Library to plot
import matplotlib.cm as colormap   # Library to plot
import matplotlib.image as mpimg       # reading images to numpy arrays
import matplotlib.pyplot as plt        # to plot any graph
import matplotlib.patches as mpatches  # to draw a circle at the mean contour
from pylab import rcParams
rcParams['figure.figsize'] = (8, 8)      # setting default size of plots

# confirm GPU is found
# print("%d device(s) found." % cuda.Device.count())           
# for ordinal in range(cuda.Device.count()):
#     dev = cuda.Device(ordinal)
#     print ("Device #%d: %s" % (ordinal, dev.name()))
# print (cuda)
# cuda.init()
# gpu_id = 1  # Change this to the desired GPU index (e.g., 1, 2, etc.)
# device = cuda.Device(gpu_id)
# context = device.make_context()
# print(f"Using GPU: {device.name()} (ID: {gpu_id})")


# ### Import Raster

# In[20]:


# replace with the correct input raster
# demfile = 'canopy_dsm_10kby10k.tif'
# demfile = "DSM_bldng_hh_reprojected_to_DEM.tif"
demfile = 'nDSM_veg_cleaned_6491.tif'
# demfile = 'boston_sample.tif'
# demfile = "/media/remap/NO_HEAT_RB/Metro_Boston/Raw/Building_Footprints_OvertureMaps/old/building_footprint_boston_height_raster.tif"
# demfile = "canopy_dsm_5kby5k.tif"
rast = demfile
src_ds=gdal.Open(rast) 
gt=src_ds.GetGeoTransform()
rb=src_ds.GetRasterBand(1)
arrayDEM = rb.ReadAsArray().astype(np.float32)  # Force correct dtype

rows = arrayDEM.shape[0]
cols = arrayDEM.shape[1]

data_flat = arrayDEM.flatten()

plt.hist(data_flat, bins=30, edgecolor='black')
plt.title('Histogram of Height Values')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

print(arrayDEM.shape)

In [ ]:
# ### Confirm Raster is Valid

# In[9]:


if np.isnan(arrayDEM).any():  
    print("Error: Nan values detected in raster. Please ensure raster has no nans prior to running this script.")
    sys.exit()


# ### CUDA Kernel

# In[21]:


#Kernel text
kernel = """
#include <math.h>

#define PI 3.1415926

__device__ void annulus_weight(float altitude, float aziinterval, float *weight) {
   float n = 90.0;
   float steprad = (360./aziinterval) * PI/180.0;
   float annulus = 91.0 - altitude;
   float w = 1.0/(2.0*PI) * sin(PI / (2.0*n)) * sin((PI * (2.0 * annulus - 1.0)) / (2.0 * n));
   *weight = steprad * w;
}


__global__ void svf_shadowcasting_2018a_gpu(float * svf_Latt, float * svf_Latt_N, float * svf_Latt_E, float * svf_Latt_S, float * svf_Latt_W, float * dsm_Latt, 
                                           float *iazimuth_Latt, float scale, 
                                           int imageW, int imageH, int rangeDist, int radius_start, int radius_step_size)
{
   #define NLM_BLOCK_RADIUS    3
   
   const long int   ix = blockDim.x * blockIdx.x + threadIdx.x;
   const long int   iy = blockDim.y * blockIdx.y + threadIdx.y;
   const float  x = (float)ix;
   const float  y = (float)iy;
   
   const float limxmin = 0;           
   const float limxmax = imageW;
   const float limymin = 0;
   const float limymax = imageH; 
   
   
   long int index4;
   
   
   if(ix>=limxmin && ix<limxmax && iy>=limymin && iy<limymax)
   {
       //The index pixel
       index4 = x + (y * imageW);
       
       float iangle[] = {6, 18, 30, 42, 54, 66, 78, 90};
       float aziinterval[] = {30, 30, 24, 24, 18, 12, 6, 1};
       float annulino[] = {0, 12, 24, 36, 48, 60, 72, 84, 90};
       
       float svf = 0;
       float svf_N = 0;
       float svf_E = 0;
       float svf_S = 0;
       float svf_W = 0;

       // this represents the total svf (and weighting) if all patches in the cardinal direction angle sector were visible (so flag = 1)

       float svf_N_max = 0;
       float svf_E_max = 0;
       float svf_S_max = 0;
       float svf_W_max = 0;

       int idx = 0;
       for (int i = 0; i<8; i++) {
           for (int j=0; j < aziinterval[i]; j++) {
               float altitude = iangle[i];
               float azimuth;

               // this was the original if statement, but this doesn't make sense to me. why would you use the previous azimuth for computations?
               //if (idx - 1 < 0) {
               //    azimuth = iazimuth_Latt[144];
               //} else {
               //    azimuth = iazimuth_Latt[idx - 1];
               //}
               // instead i just do this now:
               azimuth = iazimuth_Latt[idx];
               
               float altitude_degree = PI*altitude/180.0;
               float theta;
               
               // convert the sun azimuth (clockwise zero at North) to theta (anticlockwise, zero at east)
               if (azimuth < 90 & azimuth > 0) {
                   theta = PI*(90 - azimuth)/180.0;} 
               else { //azimuth > 180 & azimuth<360
                   theta = PI*(450 - azimuth)/180.0;}

               //if (ix == 737 & iy == 793) {
               //    printf("%f\\n", theta);
               //}  
               
               // the search radius for the SVF computing
               float sh = 0;
               float f = dsm_Latt[index4];
               float temp = 0;
               
               for(float radius = radius_start; radius < rangeDist; radius = radius + radius_step_size)
               {  
                   int rayX = int(x + radius * cos(theta));
                   int rayY = int(y - radius * sin(theta));
                   //if (ix == 0.) {
                    //   printf("%ld\\n", ix);
                    //   printf("%d\\n", rayX);
                    //   printf("%d\\n", rayY);
                    //   printf("--------");
                    //   }
                   if (rayX >= limxmax || rayX < limxmin || rayY >= limymax || rayY < limymin) 
                   {
                       break;
                   }
                   temp = 0;
                   long int index2 = rayX + (rayY * imageW);
                   // building height information
                   temp = dsm_Latt[index2] - radius*tan(altitude_degree)/scale;
                   
                   if (f < temp) {
                       f = temp;}                    
               }
               
               if (f == dsm_Latt[index4]) {
                   sh = 1;} // visible
               else {
                   sh = 0;} // not visible
               
               float weight;
               for (int k = annulino[i] + 1; k < annulino[i + 1] + 1; k++){
                   annulus_weight(k, aziinterval[i], &weight);

                   if (azimuth >= 270 || azimuth < 90) {  // total North visibility
                       svf_N_max += weight;}
                   if (azimuth >= 0 && azimuth < 180) {   // total East visibility
                       svf_E_max += weight;}
                   if (azimuth >= 90 && azimuth < 270) {  // total South visibility
                       svf_S_max += weight;}
                   if (azimuth >= 180 && azimuth < 360) { // total West visibility
                       svf_W_max += weight;}

                   weight *= sh; // multiplies weights by flags, so now we only include those patches that are visible
                   
                   svf += weight;
                   if (azimuth >= 0 && azimuth < 180) {
                       svf_E += weight;}
                   if (azimuth >= 90 && azimuth < 270) {
                       svf_S += weight;}
                   if (azimuth >= 180 && azimuth < 360) {
                       svf_W += weight;}
                   if (azimuth >= 270 || azimuth < 90) {
                       svf_N += weight;}
               }
               
               idx++;
           }}

       // normalizing svf value for each direction by the total weightage of patches in that cardinal direction sector

       svf_N = svf_N_max > 0 ? svf_N / svf_N_max : 0;
       svf_E = svf_E_max > 0 ? svf_E / svf_E_max : 0;
       svf_S = svf_S_max > 0 ? svf_S / svf_S_max : 0;
       svf_W = svf_W_max > 0 ? svf_W / svf_W_max : 0;
       
       svf_Latt[index4] = svf;
       svf_Latt_N[index4] = svf_N;
       svf_Latt_E[index4] = svf_E;
       svf_Latt_S[index4] = svf_S;
       svf_Latt_W[index4] = svf_W;
   }}
"""
#Compile and get kernel function
mod = SourceModule(kernel)
print (mod)


# ### PyCUDA Implementation

# In[22]:


def svf_shadowcasting_2018a_gpu(mod, dsmimg, scale, rangeDist, radius_start, radius_step_size):    
    '''This is the shadow casting algorithm to calculate the sky view factor
    This version will implement all the process of shadow casting and the svf
    estimation
    
    Parameter:
        mod: the module of the SVF estimation kernel
        dsmimg: the numpy array of the digital surface model
    '''
    
    index = int(0)
    iazimuth = np.hstack(np.zeros((1, 145)))
    # ialtitude = np.zeros((1, 145))
    
    azistart = np.array([0, 4, 2, 5, 8, 0, 10, 0])
    
    skyvaultaziint = np.array([12, 12, 15, 15, 20, 30, 60, 360]) #azimuth
    skyvaultaltint = np.array([6, 18, 30, 42, 54, 66, 78, 90])   #altitude
    
    for j in range(0, 8):
        for k in range(0, int(360 / skyvaultaziint[j])):
            iazimuth[index] = k * skyvaultaziint[j] + azistart[j]
            if iazimuth[index] > 360.:
                iazimuth[index] = iazimuth[index] - 360.
            index = index + 1
    
    
    # the input numpy arrays
    iazimuth = np.array(iazimuth).astype(np.float32)
    dsmimg = np.array(dsmimg).astype(np.float32)
    
    # allocate memory on the device and transfer data to GPU 
    dsm_px = cuda.mem_alloc(dsmimg.nbytes)
    iazimuth_px = cuda.mem_alloc(iazimuth.nbytes)
    
    cuda.memcpy_htod(dsm_px, dsmimg)
    cuda.memcpy_htod(iazimuth_px, iazimuth)
    
    
    height,width = dsmimg.shape
    nb_pixels = height * width
    
    
    # Set blocks et Grid sizes
    nb_ThreadsX = 32
    nb_ThreadsY = 32
    nb_blocksX = (width // nb_ThreadsX) + 1
    nb_blocksY = (height // nb_ThreadsY) + 1
    #print("Test GPU ",nb_blocksX*nb_blocksY," Blocks ",nb_ThreadsX*nb_ThreadsY," Threads/Block")
    
    
    #### Ground View Factors ####  
    # create empty array for saving 
    # svf_gpu = gpuarray.to_gpu(dsmimg)
    svf_Lattice_gpu = gpuarray.empty(dsmimg.shape, dtype=np.float32)
    svf_Lattice_gpu_N = gpuarray.empty(dsmimg.shape, dtype=np.float32)
    svf_Lattice_gpu_E = gpuarray.empty(dsmimg.shape, dtype=np.float32)
    svf_Lattice_gpu_S = gpuarray.empty(dsmimg.shape, dtype=np.float32)
    svf_Lattice_gpu_W = gpuarray.empty(dsmimg.shape, dtype=np.float32)
    
    
    #print('type of azimuth and altitude are:', type(azimuth), type(altitude))
    # the GPU function
    KNN_Mono_GPU = mod.get_function("svf_shadowcasting_2018a_gpu")
    KNN_Mono_GPU(svf_Lattice_gpu, svf_Lattice_gpu_N, svf_Lattice_gpu_E, svf_Lattice_gpu_S, svf_Lattice_gpu_W, \
                 dsm_px, \
                 iazimuth_px, \
                 np.float32(scale), \
                 np.int32(width), \
                 np.int32(height), \
                 np.int32(rangeDist), \
                 np.int32(radius_start), \
                 np.int32(radius_step_size), \
                 block=(nb_ThreadsX, nb_ThreadsY, 1), \
                 grid=(nb_blocksX, nb_blocksY))
    
    
    svfPx = np.empty_like(dsmimg)
    svfPx = svf_Lattice_gpu.get()
    svfPx_N = svf_Lattice_gpu_N.get()
    svfPx_E = svf_Lattice_gpu_E.get()
    svfPx_S = svf_Lattice_gpu_S.get()
    svfPx_W = svf_Lattice_gpu_W.get()
    svfPx = np.float32(svfPx)
    svfPx_N = np.float32(svfPx_N)
    svfPx_E = np.float32(svfPx_E)
    svfPx_S = np.float32(svfPx_S)
    svfPx_W = np.float32(svfPx_W)
    
    return svfPx, svfPx_N, svfPx_E, svfPx_S, svfPx_W

# parameters to set

rangeDist = 200
radius_start = 0
radius_step_size = 2
scale = 1

# t1 = time.time()
# result, result_N, result_E, result_S, result_W = svf_shadowcasting_2018a_gpu(mod, arrayDEM, scale, rangeDist, radius_start, radius_step_size)
# print('The time consumption is:', time.time() - t1)


# In[23]:


import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
import pycuda.gpuarray as gpuarray

# Assume `svf_shadowcasting_2018a_gpu` returns:
# SVF_total, SVF_N, SVF_E, SVF_S, SVF_W
def process_large_svf(dsm, scale, rangeDist, radius_start, radius_step_size):
    """Process SVF on a large raster by tiling it into overlapping 3800x3800 chunks."""

    height, width = dsm.shape
    TILE_SIZE = 3200
    BORDER = 400

    svf_result = np.full((height, width), -1.0, dtype=np.float32)
    svf_n_result = np.full((height, width), -1.0, dtype=np.float32)
    svf_e_result = np.full((height, width), -1.0, dtype=np.float32)
    svf_s_result = np.full((height, width), -1.0, dtype=np.float32)
    svf_w_result = np.full((height, width), -1.0, dtype=np.float32)

    num_tiles = ((height // (TILE_SIZE - 2 * BORDER)) + 1) * ((width // (TILE_SIZE - 2 * BORDER)) + 1)
    tile_count = 0

    for i in range(0, height, TILE_SIZE - 2 * BORDER):
        for j in range(0, width, TILE_SIZE - 2 * BORDER):
            i_start = max(i - BORDER, 0)
            i_end = min(i + TILE_SIZE + BORDER, height)
            j_start = max(j - BORDER, 0)
            j_end = min(j + TILE_SIZE + BORDER, width)

            dsm_tile = dsm[i_start:i_end, j_start:j_end]

            # Get all 5 directional SVF outputs
            tile_svf, tile_svf_n, tile_svf_e, tile_svf_s, tile_svf_w = svf_shadowcasting_2018a_gpu(
                mod, dsm_tile, scale, rangeDist, radius_start, radius_step_size
            )

            valid_i_start = BORDER if i > 0 else 0
            valid_j_start = BORDER if j > 0 else 0
            valid_i_end = TILE_SIZE if i + TILE_SIZE < height else i_end - i_start
            valid_j_end = TILE_SIZE if j + TILE_SIZE < width else j_end - j_start

            svf_valid = tile_svf[valid_i_start:valid_i_end, valid_j_start:valid_j_end]
            n_valid = tile_svf_n[valid_i_start:valid_i_end, valid_j_start:valid_j_end]
            e_valid = tile_svf_e[valid_i_start:valid_i_end, valid_j_start:valid_j_end]
            s_valid = tile_svf_s[valid_i_start:valid_i_end, valid_j_start:valid_j_end]
            w_valid = tile_svf_w[valid_i_start:valid_i_end, valid_j_start:valid_j_end]

            result_i_start = i
            result_j_start = j
            result_i_end = result_i_start + svf_valid.shape[0]
            result_j_end = result_j_start + svf_valid.shape[1]

            svf_result[result_i_start:result_i_end, result_j_start:result_j_end] = svf_valid
            svf_n_result[result_i_start:result_i_end, result_j_start:result_j_end] = n_valid
            svf_e_result[result_i_start:result_i_end, result_j_start:result_j_end] = e_valid
            svf_s_result[result_i_start:result_i_end, result_j_start:result_j_end] = s_valid
            svf_w_result[result_i_start:result_i_end, result_j_start:result_j_end] = w_valid

            tile_count += 1
            percent_done = (tile_count / num_tiles) * 100
            print(f"Progress: {tile_count}/{num_tiles} tiles processed, {percent_done:.2f}% completed.", flush=True)

    return svf_result, svf_n_result, svf_e_result, svf_s_result, svf_w_result

# Example usage
dsm = arrayDEM
scale = 1
rangeDist = 200
radius_start = 0
radius_step_size = 2

t_start = time.time()
svf_total, svf_north, svf_east, svf_south, svf_west = process_large_svf(dsm, scale, rangeDist, radius_start, radius_step_size)
print('The time consumption is:', time.time() - t_start)

# np.save("shadowcasting_GPU_output.npy", svf_total)
# np.save("shadowcasting_GPU_output_north.npy", svf_north)
# np.save("shadowcasting_GPU_output_east.npy", svf_east)
# np.save("shadowcasting_GPU_output_south.npy", svf_south)
# np.save("shadowcasting_GPU_output_west.npy", svf_west)

# Save or visualize the output SVF grid
data_flat = svf_total.flatten()

plt.hist(data_flat, bins=100, edgecolor='black')
plt.title('Histogram of SVF Values')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()


# ### Convert to Geotiff and save raster

# In[24]:


# Copy metadata from original tif
with rasterio.open(demfile) as src:
    meta = src.meta.copy()

canopy = True

# Update metadata for output
meta.update({
    "dtype": svf_total.dtype,
    "count": 1
})

# Output filenames for each array
if canopy:
    outputs = {
        "shadowcasting_gpu_svf_total_can.tif": svf_total,
        "shadowcasting_gpu_svf_north_can.tif": svf_north,
        "shadowcasting_gpu_svf_east_can.tif": svf_east,
        "shadowcasting_gpu_svf_south_can.tif": svf_south,
        "shadowcasting_gpu_svf_west_can.tif": svf_west
    }
else:
    outputs = {
        "shadowcasting_gpu_svf_total.tif": svf_total,
        "shadowcasting_gpu_svf_north.tif": svf_north,
        "shadowcasting_gpu_svf_east.tif": svf_east,
        "shadowcasting_gpu_svf_south.tif": svf_south,
        "shadowcasting_gpu_svf_west.tif": svf_west
    }

# Write each array to corresponding tif
for filename, data in outputs.items():
    with rasterio.open(filename, "w", **meta) as dst:
        dst.write(data, 1)  # write to the first band


# ### Validation

# #### Confirm SVF values are between 0 and 1

# In[9]:


# print("Minimum value in total SVF array: " + str(np.min(result)))
# print("Maximum value in total SVF array: " + str(np.max(result)))

# print("Minimum value in North SVF array: " + str(np.min(result_N)))
# print("Maximum value in North SVF array: " + str(np.max(result_N)))

# print("Minimum value in East SVF array: " + str(np.min(result_E)))
# print("Maximum value in East SVF array: " + str(np.max(result_E)))

# print("Minimum value in South SVF array: " + str(np.min(result_S)))
# print("Maximum value in South SVF array: " + str(np.max(result_S)))

# print("Minimum value in West SVF array: " + str(np.min(result_W)))
# print("Maximum value in West SVF array: " + str(np.max(result_W)))


# #### Confirm that cardinal direction calculated values are similar to total SVF values

# In[10]:


# SVF_original = result
# SVF_calculated = (result_N + result_E + result_S + result_W)/4

# diff = SVF_original - SVF_calculated

# print("Max difference: " + str(np.max(np.abs(diff))))
# print("Average difference: " + str(np.mean(np.abs(diff))))
# num_pixels = SVF_original.shape[0]*SVF_original.shape[1]
# print("Fraction of pixels with difference >= 0.01: " + str(np.round((diff[np.abs(diff) >= 0.01].shape[0])/num_pixels,3)))
# print("Fraction of pixels with difference >= 0.001: " +  str(np.round((diff[np.abs(diff) >= 0.001].shape[0])/num_pixels,3)))


# #### Histogram of SVF values

# In[11]:


# data_flat = SVF_calculated.flatten()

# plt.hist(data_flat, bins=30, edgecolor='black')
# plt.title('Histogram of SVF Values')
# plt.xlabel('Value')
# plt.ylabel('Frequency')
# plt.show()


# ### Pickled Results for Comparison
# - Uncomment this section when we want to pickle the results for comparison to other algorithms

# In[8]:


# import pickle

# with open('shadow_gpu_SVF_calc.pkl', 'wb') as f:
#     pickle.dump(SVF_calculated, f)

# with open('shadow_gpu_SVF_N.pkl', 'wb') as f:
#     pickle.dump(result_N, f)

# with open('shadow_gpu_SVF_E.pkl', 'wb') as f:
#     pickle.dump(result_E, f)

# with open('shadow_gpu_SVF_S.pkl', 'wb') as f:
#     pickle.dump(result_S, f)

# with open('shadow_gpu_SVF_W.pkl', 'wb') as f:
#     pickle.dump(result_W, f)
